In [36]:
import numpy as np
import pandas as pd
from scipy import special
from random import sample
import itertools
from datetime import timedelta
from IPython.display import display

In [37]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [42]:
fraude_df = pd.read_csv("Dados_fraude_modificados.csv")

In [44]:
onehotencoded = [feature for feature in fraude_df.columns if "pais" not in feature and feature !="fraude"]

X = fraude_df[ [col for col in fraude_df.columns if col != "fraude"]]
Y = fraude_df["fraude"]

In [46]:
X.head()

,media_id,tempo,pais_Afghanistan,pais_Albania,pais_Algeria,pais_Angola,pais_Antigua and Barbuda,pais_Argentina,pais_Armenia,pais_Australia,...,pais_United States,pais_Uruguay,pais_Uzbekistan,pais_Vanuatu,pais_Venezuela,pais_Viet Nam,pais_Virgin Islands (U.S.),pais_Yemen,pais_Zambia,pais_Zimbabwe
0,-4.881535,-0.197359,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-4.881535,-0.161745,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0.199201,-4.801150,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,-4.881535,0.101268,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-4.881535,-0.009980,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [47]:
X.index = range(len(X))
Y.index = range(len(Y))

In [48]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier()
_ = clf.fit(X,Y)

features = pd.DataFrame()
features['feature'] = X.columns
features['importancia'] = clf.feature_importances_

features [features.importancia > np.mean(features.importancia)].sort_values(by="importancia", ascending=False)

/home/sarinha/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


,feature,importancia
1,tempo,0.735548
0,media_id,0.258814


In [49]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score

In [50]:
X_dev, X_eval, Y_dev, Y_eval = train_test_split(X, Y, train_size = 0.75, random_state = 23)
Y_dev = Y_dev.apply(lambda x: 1 if x == 0 else -1)
Y_eval = Y_eval.apply(lambda x: 1 if x == 0 else -1)

In [ ]:
IF = IsolationForest(max_samples="auto",contamination = 0.095, random_state=1)
param_grid = {"n_estimators": [100, 150, 200, 250, 300, 350, 400, 450, 500]}

clf = GridSearchCV(IF,
                   param_grid,
                   cv=2,
                   n_jobs=3,
                   scoring="f1",
                   verbose=10)

_ = clf.fit(X_dev, Y_dev)


In [ ]:
IF_best = clf.best_estimator_
Y_predicted = IF_best.predict(X_eval)
roc_auc = roc_auc_score(Y_eval,Y_predicted)

print(classification_report(Y_eval,
                           Y_predicted,
                           target_names=["anomalo","normal"]))
print("Área sob a curva ROC: {:0.3f}".format(roc_auc))

IF_probs = IF_best.decision_function(X_eval)
fpr, tpr, thresold = roc_curve(Y_eval, IF_probs)

plt.plot(fpr, tpr, lw = 1)

plt.plot([0,1],[0,1], '--', color=(0.6, 0.6, 0.6), label="sorte")
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])

plt.xlabel("Taxa de positivos falsos", fontsize=15)
plt.ylabel("Taxa de positivos verdadeiros", fontsize=15)
plt.title("Curva ROC", fontsize=12)

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
cnf_matrix_IF = confusion_matrix(Y_eval, Y_predicted)
print(cnf_matrix_IF)

In [ ]:
X["fraude"] = Y

X_anomalias = X[X['fraude'] == 1]
X_normal = X[X['fraude'] == 0]

train_idxs = sample(list(X_normal.index), int(0.7* X_normal.shape[0]))
X_train = X_normal.loc[train_idxs]

X_testing = X_normal.drop(train_idxs)

X_testing = pd.concat([X_testing, X_anomalias], axis=0)

X_train = X_train.sample(frac=1).reset_index(drop=True)
X_testing = X_testing.sample(frac=1).reset_index(drop=True)

Y_testing = X_testing['fraude']
X_testing = X_testing[ [ col for col in X_testing.columns if col != 'fraude']]

X_cv, X_eval, Y_cv, Y_eval = train_test_split(X_testing, Y_testing, train_size = 0.7, random_state=23)

Y_cv = Y_cv.apply(lambda x: 1 if x==0 else -1)
Y_eval = Y_eval.apply(lambda x: 1 if x ==0 else -1)

X_train = X_train[ [col for col in X_testing.columns if col !='fraude']]

In [ ]:
param_grid = {"nu": np.linspace(0.001, 0.01, 5), "gamma": [0.01, 0.03, 0.1, 0.3]}
param_list = list(itertools.product(param_grid["nu"],param_grid["gamma"]))

In [ ]:
from sklearn.svm import OneClassSVM

def random_gridsearch(param_list, n_sample, X_train, X_cv, Y_cv):
    
    f1_max = 0
    param_list = sample(param_list, n_sample)
    remaining_fits = n_sample
    for params in param_list:
        nu, gamma = params
        OCSVM = OneClassSVM(kernel="rbf", nu = nu, gamma=gamma, random_state=1  )
        print('Fitando modelo...')
        OCSVM.fit(X_train)
        
        cv_preds = OCSVM.predict(X_cv)
        f1 = f1_score(Y_cv.values, cv_preds, average="binary")
        
        if f1 > f1_max:
            f1_max = f1
            OCSVN_best = OCSVM
        
            remaining_fits -=1
        
    return f1_max, OCSVM
    
f1_max, OCSVM_best = random_gridsearch(param_list, 10, X_train, X_cv, Y_cv)

In [ ]:
OCSVM_best.fit(X_train)
Y_predicted = OCSVM_best.predict(X_eval)
roc_auc = roc_auc_score(Y_eval,Y_predicted)

print(classification_report(Y_eval,
                           Y_predicted,
                           target_names=['anormais','normais']))
print('Área sob a curva ROC: {:0.3f}'.format(roc_auc))

OCSVM_probs = OCSVM_best.decision_function(X_eval)
fpr, tpr, thresholds = roc_curve(Y_eval, OCSVM_probs)

plt.plot(fpr,tpr, lw=1, label='ROC')
plt.plot([0,1],[0,1],'--', color=(0.6, 0.6, 0.6),label='sorte')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])

plt.xlabel('Taxa de positivos falsos', fontsize=15)
plt.ylabel('Taxa de positivos verdadeiros',fontsize=15)
plt.title('Curva ROC')

plt.show()

In [ ]:
print(confusion_matrix(Y_eval,Y_predicted))